In [1]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

import pandas as pd

df = pd.read_csv('Retailer Sales Data.csv')
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d/%m/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%d/%m/%Y')


# ETL PROCESS

In [12]:

#! Handling the order date missing values

print( "Number of missing values on 'Ship Date' and 'Order Date':", df[df['Ship Date'].isnull() & df['Order Date'].isnull()].shape[0])

df['Shipping_days'] = df['Ship Date'] - df['Order Date'] 


# PRINT THE average of (df['Shipping_days']) 
avg_shipping_days = df['Shipping_days'].mean()

# Se "Order Date" è mancante, sostituiscilo con "Ship Date" meno la media dei giorni di spedizione
df.loc[df['Order Date'].isnull(), 'Order Date'] = df['Ship Date'] - pd.to_timedelta(avg_shipping_days, unit='D')


# Drop 'Order Date' row if Nan
df = df[df['Order Date'].notna()]

#df['Order Date'].isna().value_counts()
"""
False    9301
True      499

False    9786
True       14
"""

#  Convert Order Date to `year-month` level
df['Year-Month'] = df['Order Date'].dt.to_period('M')

print("Missing months of sales:",
    pd.period_range(start=df['Year-Month'].min(), 
                                 end=df['Year-Month'].max(), 
                                 freq='M').difference(df['Year-Month']))

del avg_shipping_days


Number of missing values on 'Ship Date' and 'Order Date': 0
Missing months of sales: PeriodIndex([], dtype='period[M]')


In [13]:

#! Handle missing values of Sales
print( "Number of missing values on 'Sales' and 'Sub-Category':", df[df['Sales'].isnull() & df['Sub-Category'].isnull()].shape[0])

# create a new df called "product_price" and group by Sub-Category' and get the average Sales
product_price_sub_cat = df.groupby('Sub-Category')['Sales'].mean().reset_index()
product_price_cat = df.groupby('Category')['Sales'].mean().reset_index()

# if sales is missing assign the average price of the sub-category to sales

df['Sales'] = df['Sales'].fillna(df['Sub-Category'].map(product_price_sub_cat.set_index('Sub-Category')['Sales']))
df['Sales'] = df['Sales'].fillna(df['Category'].map(product_price_cat.set_index('Category')['Sales']))
df = df[df['Sales'].notna()]

del product_price_sub_cat
del product_price_cat

Number of missing values on 'Sales' and 'Sub-Category': 23


In [16]:
df['State'].isna().value_counts()

False    9296
True      488
Name: State, dtype: int64

In [38]:
# Check if count cUSTOMER ID has more than one 'Customer Name'
(df.groupby('Customer ID')['Customer Name'].nunique() > 1).sum()

state_fill_dict = df.dropna(subset=['State']).groupby('Customer ID')['State'].first().to_dict()

# Passo 2: Usa il dizionario per riempire i valori mancanti nella colonna 'State'
df['State'] = df.apply(
    lambda row: state_fill_dict.get(row['Customer ID'], row['State']) 
    if pd.isna(row['State']) 
    else row['State'], 
    axis=1
)

city_fill_dict = df.dropna(subset=['State', 'Country']).groupby('Country')['State'].first().to_dict()

# Passo 2: Riempi i valori mancanti nella colonna 'State' basati su 'Country'
df['State'] = df.apply(
    lambda row: city_fill_dict.get(row['Country'], row['State']) 
    if pd.isna(row['State']) and not pd.isna(row['Country']) 
    else row['State'], 
    axis=1
)

df = df[df['State'].notna()]

df['State'].isna().value_counts()

del city_fill_dict
del state_fill_dict


False    9781
Name: State, dtype: int64

# MODELS

In [14]:
print(df['Year-Month'].min(), df['Year-Month'].max())

2015-01 2018-12


In [18]:
df.columns


Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Shipping_days', 'Year-Month'],
      dtype='object')